In [1]:
# Cell 1: Import necessary libraries
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print("Libraries loaded successfully.")

Libraries loaded successfully.


In [3]:
# Cell 2: Load images from folders
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

# Define folder paths
low_light_folder = r'E:\Project\archive\lol_dataset\eval15\low'
enhanced_folder = r'E:\Project\archive\lol_dataset\eval15\high'

# Load the images from the respective folders
low_light_images = load_images_from_folder(low_light_folder)
enhanced_images = load_images_from_folder(enhanced_folder)

print("Images loaded successfully.")

Images loaded successfully.


In [5]:
# Cell 3: Convert images to NumPy arrays and normalize
low_light_images = np.array(low_light_images) / 255.0
enhanced_images = np.array(enhanced_images) / 255.0

print(f"Low-light images shape: {low_light_images.shape}")
print(f"Enhanced images shape: {enhanced_images.shape}")

Low-light images shape: (15, 400, 600, 3)
Enhanced images shape: (15, 400, 600, 3)


In [7]:
# Cell 4: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(low_light_images, enhanced_images, test_size=0.2, random_state=42)

# Debugging: Check the size of the training and testing sets
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (12, 400, 600, 3)
Test set shape: (3, 400, 600, 3)


In [9]:
# Cell 5: Define U-Net model
def build_unet():
    inputs = Input(shape=(256, 256, 3))

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Decoder
    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = concatenate([up5, conv3], axis=-1)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = concatenate([up6, conv2], axis=-1)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = concatenate([up7, conv1], axis=-1)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    outputs = Conv2D(3, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build the model and print summary
unet_model = build_unet()
unet_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_15 (Conv2D)            │ (None, 256, 256, 64)      │           1,792 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_16 (Conv2D)            │ (None, 256, 256, 64)      │          36,928 │ conv2d_15[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 128, 128, 64)      │               0 │ conv2d_16[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_17 (Conv2D)            │ (None, 128, 128, 128)     │          73,856 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_18 (Conv2D)            │ (None, 128, 128, 128)     │         147,584 │ conv2d_17[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 64, 64, 128)       │               0 │ conv2d_18[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_19 (Conv2D)            │ (None, 64, 64, 256)       │         295,168 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_20 (Conv2D)            │ (None, 64, 64, 256)       │         590,080 │ conv2d_19[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_5               │ (None, 32, 32, 256)       │               0 │ conv2d_20[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_21 (Conv2D)            │ (None, 32, 32, 512)       │       1,180,160 │ max_pooling2d_5[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_22 (Conv2D)            │ (None, 32, 32, 512)       │       2,359,808 │ conv2d_21[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_3               │ (None, 64, 64, 512)       │               0 │ conv2d_22[0][0]            │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_3 (Concatenate)   │ (None, 64, 64, 768)       │               0 │ up_sampling2d_3[0][0],     │
│                               │                           │                 │ conv2d_20[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_23 (Conv2D)            │ (None, 64, 64, 256)       │       1,769,72

 Total params: 7,783,043 (29.69 MB)

 Trainable params: 7,783,043 (29.69 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Cell 6: Resize images to target shape and ensure 3-channel color format
def resize_images(data, target_shape=(256, 256)):
    resized_data = np.array([cv2.resize(img, target_shape) for img in data])
    # Ensure 3 channels (in case of grayscale or inconsistent channels)
    if resized_data.shape[-1] != 3:
        resized_data = np.repeat(resized_data[..., np.newaxis], 3, axis=-1)
    return resized_data

# Resize training and test sets
X_train_resized = resize_images(X_train)
X_test_resized = resize_images(X_test)
y_train_resized = resize_images(y_train)
y_test_resized = resize_images(y_test)

print("Image resizing completed.")
print(f"X_train_resized shape: {X_train_resized.shape}")
print(f"y_train_resized shape: {y_train_resized.shape}")
print(f"X_test_resized shape: {X_test_resized.shape}")
print(f"y_test_resized shape: {y_test_resized.shape}")

Image resizing completed.
X_train_resized shape: (12, 256, 256, 3)
y_train_resized shape: (12, 256, 256, 3)
X_test_resized shape: (3, 256, 256, 3)
y_test_resized shape: (3, 256, 256, 3)


In [13]:
# Cell 7: Train the model, ensuring the input shapes match
# Double-check input shape before training
assert X_train_resized.shape[1:] == (256, 256, 3), "Input shape does not match model's expected input."
assert y_train_resized.shape[1:] == (256, 256, 3), "Output shape does not match model's expected output."

history = unet_model.fit(
    X_train_resized, y_train_resized,
    epochs=10, batch_size=8,
    validation_data=(X_test_resized, y_test_resized),
    verbose=1
)

# Print training history to verify
print("Training completed.")
print("History:", history.history)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 48s 20s/step - accuracy: 0.1745 - loss: 0.6673 - val_accuracy: 0.6054 - val_loss: 0.1274
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 47s 17s/step - accuracy: 0.3967 - loss: 0.1136 - val_accuracy: 0.2666 - val_loss: 0.2428
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 51s 20s/step - accuracy: 0.3748 - loss: 0.2360 - val_accuracy: 0.5428 - val_loss: 0.0632
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 50s 18s/step - accuracy: 0.4624 - loss: 0.0563 - val_accuracy: 0.6152 - val_loss: 0.0325
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 53s 19s/step - accuracy: 0.5370 - loss: 0.0430 - val_accuracy: 0.6118 - val_loss: 0.0408
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 52s 18s/step - accuracy: 0.4363 - loss: 0.0365 - val_accuracy: 0.1152 - val_loss: 0.0204
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 53s 19s/step - accuracy: 0.0956 - loss: 0.0227 - val_accuracy: 0.1077 - val_loss: 0.0301
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 50s 18s/step - accuracy: 0.0889 - loss: 0.0319 - val_accuracy: 0.1071 - val_loss: 0.0314
